# Browser-Use SDK を使用したライブビュー Browser ツール

## 概要

このチュートリアルでは、Browser-Use を使用して Amazon Bedrock AgentCore Browser ツールを操作し、ブラウザをライブで表示する方法を学びます。


### チュートリアル詳細


| 項目         | 詳細                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ       | 会話型                                                                   |
| エージェントタイプ          | シングル                                                                           |
| エージェントフレームワーク   | Browser-Use                                                                      |
| LLM モデル           | Anthropic Claude 3.7 Sonnet                                                      |
| チュートリアルコンポーネント | Browser-Use を使用した Browser ツールのライブ操作                 |
| チュートリアル分野   | 垂直統合                                                                         |
| 難易度  | 簡単                                                                             |
| 使用 SDK            | Amazon Bedrock AgentCore Python SDK、Browser-Use                                  |

### チュートリアルアーキテクチャ

このチュートリアルでは、Browser-Use と Browser ツールを使用してブラウザをライブで表示する方法を説明します。  

この例では、Browser-Use エージェントに自然言語の指示を送り、Bedrock AgentCore Browser 上でタスクを実行しながらブラウザをライブで表示します。

<div style="text-align:left">
    <img src="images/browser-tool.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Browser ツールの使用とライブビュー表示 
* Browser-Use と Browser ツールの連携

## 前提条件

### このチュートリアルを実行するには、以下が必要です：
* Python 3.11+
* AWS 認証情報。IAM ロール/ユーザーには以下の権限が必要です：https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/browser-onboarding.html#browser-credentials-config
* Amazon Bedrock AgentCore SDK
* Browser-Use SDK 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

### browser-use の問題を修正するために以下のパッチスクリプトを実行してください

In [ ]:
%%writefile patch_browser_use.py

#!/usr/bin/env python3
"""Auto-detect and patch browser_use session.py"""

import os
import shutil
import sys
from pathlib import Path

def find_browser_use_path():
    """Automatically find the browser_use installation path"""
    try:
        import browser_use
        browser_use_path = Path(browser_use.__file__).parent
        session_file = browser_use_path / "browser" / "session.py"
        return str(session_file)
    except ImportError:
        print("❌ browser_use not installed. Install with: pip install browser-use")
        return None

def patch_browser_use():
    # Auto-detect file path
    file_path = find_browser_use_path()
    if not file_path:
        return False
    
    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return False
    
    print(f"📁 Found browser_use at: {file_path}")
    
    # Create backup
    backup_path = file_path + ".backup"
    if not os.path.exists(backup_path):
        shutil.copy2(file_path, backup_path)
        print(f"💾 Created backup: {backup_path}")
    else:
        print(f"📋 Backup already exists: {backup_path}")
    
    # Read file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Replacement 1: Add headers check after cdp_url check
    old1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True"
    new1 = "if not cdp_url:\n\t\t\tprofile_kwargs['is_local'] = True\n\n\t\tif headers:\n\t\t\tprofile_kwargs['headers'] = headers"
    
    if old1 in content and "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" not in content:
        content = content.replace(old1, new1)
        print("✅ Added headers check")
    elif "if headers:\n\t\t\tprofile_kwargs['headers'] = headers" in content:
        print("✅ Headers check already exists")
    else:
        print("⚠️ Headers check pattern not found")
    
    # Replacement 2: Add headers to CDPClient
    old2 = "self._cdp_client_root = CDPClient(self.cdp_url)"
    new2 = "self._cdp_client_root = CDPClient(self.cdp_url,  additional_headers=self.browser_profile.headers)"
    
    if old2 in content:
        content = content.replace(old2, new2)
        print("✅ Added headers to CDPClient")
    elif "additional_headers=self.browser_profile.headers" in content:
        print("✅ CDPClient headers already exists")
    else:
        print("⚠️ CDPClient pattern not found")
    
    # Write back
    with open(file_path, 'w') as f:
        f.write(content)
    
    print("🎉 Patching complete!")
    return True

if __name__ == "__main__":
    success = patch_browser_use()
    sys.exit(0 if success else 1)

In [ ]:
# Run the python script to apply the patch for browser-use
!python patch_browser_use.py

In [ ]:
# Restart the Kernal 
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

## ライブビュー付きで Bedrock AgentCore Browser ツールを Browser-Use で使用

ここでは、Amazon DCV SDK 経由で Bedrock AgentCore Browser ツールに接続するためのヘルパークラス `BrowserViewerServer` を使用します。



In [ ]:
%%writefile live_view_with_browser_use.py
from browser_use import Agent
# from browser_use.browser.session import BrowserSession
from browser_use import Browser, BrowserProfile
from bedrock_agentcore.tools.browser_client import BrowserClient
# from browser_use.browser import BrowserProfile
# from langchain_aws import ChatBedrockConverse
from browser_use.llm import ChatAnthropicBedrock, ChatAWSBedrock
from rich.console import Console
from rich.panel import Panel
from contextlib import suppress
import argparse
import sys
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer
import asyncio
from boto3.session import Session

console = Console()

boto_session = Session()
region = boto_session.region_name


async def run_browser_task(
    browser_session: Browser, bedrock_chat: ChatAnthropicBedrock, task: str
) -> None:
    """
    browser_use を使用してブラウザ自動化タスクを実行します

    Args:
        browser_session: 再利用する既存のブラウザセッション
        bedrock_chat: Bedrock チャットモデルインスタンス
        task: エージェントに対する自然言語タスク
    """
    try:
        # タスク実行を表示
        console.print(f"\n[bold blue]タスクを実行中：[/bold blue] {task}")

        # エージェントを作成して実行
        agent = Agent(task=task, llm=bedrock_chat, browser_session=browser_session)

        # 進捗インジケーターと共に実行
        with console.status(
            "[bold green]ブラウザ自動化を実行中...[/bold green]", spinner="dots"
        ):
            await agent.run()

        console.print("[bold green]タスクが正常に完了しました！[/bold green]")

    except Exception as e:
        console.print(f"[bold red]タスク実行中にエラー：[/bold red] {str(e)}")
        import traceback

        if console.is_terminal:
            traceback.print_exc()


async def live_view_with_browser_use(prompt, region="us-west-2"):
    """
    エージェント自動化によるライブブラウザ表示を実演するメイン関数。

    ワークフロー:
    1. us-west-2 リージョンで Amazon Bedrock AgentCore ブラウザクライアントを作成
    2. ブラウザの初期化を待機（10 秒の必須遅延）
    3. ポート 8000 でブラウザ制御付きの DCV ベースライブビューワーサーバーを開始
    4. 複数のディスプレイサイズオプションを設定（720p から 1440p）
    5. CDP WebSocket 経由で AI エージェント自動化用のブラウザセッションを確立
    6. Claude 3.7 Sonnet モデルを使用して AI 駆動タスクを実行
    7. すべてのセッションを適切に閉じてブラウザクライアントを停止

    機能:
    - Web インターフェース経由のリアルタイムブラウザ表示
    - 手動での制御取得/解放機能
    - browser-use ライブラリによる AI 自動化
    - 設定可能なディスプレイレイアウトとサイズ
    """
    console.print(
        Panel(
            "[bold cyan]ブラウザライブビューワー[/bold cyan]\n\n"
            "このデモで説明する内容：\n"
            "・DCV によるライブブラウザ表示\n"
            "・設定可能なディスプレイサイズ（900x800 に限定されません）\n"
            "・適切なディスプレイレイアウトコールバック\n\n"
            "[yellow]注意：Amazon DCV SDK ファイルが必要です[/yellow]",
            title="ブラウザライブビューワー",
            border_style="blue",
        )
    )

    try:
        # ステップ 1: ブラウザセッションを作成
        client = BrowserClient(region)
        client.start()
        
        ws_url, headers = client.generate_ws_headers()

        # ステップ 2: ビューワーサーバーを開始
        console.print("\n[cyan]ステップ 3: ビューワーサーバーを開始中...[/cyan]")
        viewer = BrowserViewerServer(client, port=8000)
        viewer_url = viewer.start(open_browser=True)

        # ステップ 3: 機能を表示
        console.print("\n[bold green]ビューワー機能：[/bold green]")
        console.print(
            "・デフォルトディスプレイ：1600x900（displayLayout コールバックで設定）"
        )
        console.print("・サイズオプション：720p、900p、1080p、1440p")
        console.print("・リアルタイムディスプレイ更新")
        console.print("・制御の取得/解放機能")

        console.print("\n[yellow]停止するには Ctrl+C を押してください[/yellow]")

        # ステップ 4: browser-use を使用してブラウザを操作
        # 永続的なブラウザセッションとモデルを作成
        browser_session = None
        bedrock_chat = None

        try:
            # ヘッダー付きでブラウザプロファイルを作成
            browser_profile = BrowserProfile(
                headers=headers,
                timeout=1500000,  # 150 秒のタイムアウト
            )

            # CDP URL と keep_alive=True で永続化用のブラウザセッションを作成
            browser_session = Browser(
                cdp_url=ws_url,
                browser_profile=browser_profile,
                keep_alive=True,  # タスク間でブラウザを維持
            )

            # ブラウザセッションを初期化
            console.print("[cyan]ブラウザセッションを初期化中...[/cyan]")
            await browser_session.start()

            # ChatBedrockConverse を一度だけ作成
            bedrock_chat = ChatAnthropicBedrock(
                model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
                aws_region=region,
            )

            console.print(
                "[green]ブラウザセッションが初期化され、タスクの準備ができました[/green]\n"
            )

            task = prompt

            await run_browser_task(browser_session, bedrock_chat, task)

        finally:
            # ブラウザセッションを閉じる
            if browser_session:
                console.print("\n[yellow]ブラウザセッションを閉じています...[/yellow]")
                with suppress(Exception):
                    await browser_session.close()
                console.print("[green]ブラウザセッションを閉じました[/green]")
   
    except Exception as e:
        console.print(f"\n[red]エラー: {e}[/red]")
        import traceback
        traceback.print_exc()
    finally:
        console.print("\n\n[yellow]シャットダウン中...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("ブラウザセッションを終了しました")


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--prompt", required=True, help="ブラウザ検索の指示")
    parser.add_argument("--region", default="us-west-2", help="AWS リージョン")
    args = parser.parse_args()

    asyncio.run(live_view_with_browser_use(
        args.prompt, args.region
    ))

#### スクリプトの実行
以下のスクリプトを実行してください。スクリプトが実行された後（プロンプトの複雑さに応じて時間がかかります）、出力をスクロールして実行されたアクションの結果を確認してください。 

In [ ]:
!python live_view_with_browser_use.py --prompt "Search for macbooks on amazon.com and extract the details of the first one" 

## 裏側で何が起きていたか？ 
* Browser クライアントをインスタンス化してセッションを開始しました
* その後、`BrowserViewerServer` を使用してブラウザセッションに接続し、セッションをローカルで表示しました
* Browser-Use Agent を作成し、ブラウザセッションの詳細を渡しました
* その後、Browser-Use エージェントに自然言語の指示を送り、アクションをライブで確認しました


# おめでとうございます！ 